In [6]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import StructField, StructType, DoubleType,MapType, StringType,ArrayType, FloatType, TimestampType, IntegerType
from cerebralcortex.core.datatypes import DataStream
from cerebralcortex.core.metadata_manager.stream.metadata import Metadata, DataDescriptor, \
ModuleMetadata
from scipy import signal
from scipy.interpolate import interp1d
from cerebralcortex import Kernel
from scipy.stats import skew,kurtosis,mode
from collections import Counter
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
import pickle
CC = Kernel("/home/jupyter/cc3_conf/", study_name='mperf')

In [ ]:
def get_shortened_data(data_windowed,minutes=2000,window_size=20,training_percentage=.8):
    schema = data_windowed.schema
    @pandas_udf(schema, PandasUDFType.GROUPED_MAP)
    def get_user_data(df):
        df = df[int(df.shape[0]*training_percentage):]
        return df
    data_windowed = data_windowed.groupBy(['user','day']).apply(get_user_data)
    n = int(minutes*60/window_size)
    @pandas_udf(schema, PandasUDFType.GROUPED_MAP)
    def get_user_data_sort(df):
        df = df.sort_values('timestamp').reset_index(drop=True)
        if n>=df.shape[0]//2:
            m = df.shape[0]//2
        else:
            m = n
        df = df[int(df.shape[0]-m):]
        return df
    data_windowed = data_windowed.groupBy('user').apply(get_user_data_sort)
    return data_windowed
    
activities = ['stationery']
window_size = 20
training_percentage = .8
activity = activities[0]
for activity in activities:
    print('-'*20+activity+'-'*20)
    directory = './data/'+str(window_size)+'/'+activity+'/'
    if not os.path.isdir(directory+'training'):
        os.makedirs(directory+'training')
    if not os.path.isdir(directory+'testing'):
        os.makedirs(directory+'testing')
    stream_name = 'org.md2k.feature.motionsensehrv.decoded.rightwrist.all.activity.all.three.'+str(window_size)+'.secs.'+str(activity)
    users = CC.get_stream(stream_name).select('user').distinct().toPandas()['user'].values
    for i,user_id in enumerate(users):
        data = CC.get_stream(stream_name,user_id=user_id)
        if activity=='stationery':
            data = get_shortened_data(data,minutes=1000,window_size=20)
        data = data.toPandas()
        data  = data.sort_values('timestamp').reset_index(drop=True)
#         if activity == 'stationery':
#             data = data[int(data.shape[0]*training_percentage):] 
        data['time'] = data['start'].apply(lambda a:a.timestamp())
        data['data'] = data['data'].apply(lambda a:np.array(a).reshape(1,-1,3))
        if activity =='stationery':
            testing_data  = data
            pickle.dump(testing_data,open(directory+'testing/'+str(user_id),'wb'))
        else:
            training_data = data[:int(data.shape[0]*training_percentage)] 
            testing_data = data[int(data.shape[0]*training_percentage):] 
            pickle.dump(training_data,open(directory+'training/'+str(user_id),'wb'))
            pickle.dump(testing_data,open(directory+'testing/'+str(user_id),'wb'))
        print(i,testing_data.shape[0],end=',')
    print()

--------------------stationery--------------------
0 3000,1 3000,2 3000,3 19,4 1,5 3000,6 65,7 3000,8 3000,9 3000,10 3000,11 3000,12 3000,13 907,14 795,15 3000,16 3000,17 3000,18 3000,19 3000,20 3000,21 3000,22 3000,23 3000,24 3000,25 3000,26 3000,27 20,28 3000,29 3000,30 3000,31 3000,32 3000,33 3000,34 3000,35 3000,36 3000,37 3000,38 412,39 3000,40 3000,41 3000,42 3000,43 3000,44 3000,45 3000,153 3000,154 3000,155 3000,156 3000,157 121,158 3000,159 3000,160 3000,161 3000,162 3000,163 3000,164 5,165 3000,166 3000,167 3000,168 2131,169 3000,170 3000,171 3000,172 3000,173 3000,174 3000,175 1537,176 3000,177 3000,178 3000,179 3000,180 2205,181 3000,182 3000,183 3000,184 3000,185 3000,186 3000,187 3000,188 3000,189 1753,190 3000,191 3000,192 3000,193 3000,194 3000,195 3000,196 3000,197 3000,198 337,199 3000,200 1747,201 3000,202 3000,203 3000,204 3000,205 396,206 3000,207 3000,208 3000,209 3000,210 3000,211 49,212 3000,213 3000,214 822,215 50,216 3000,217 3000,218 103,219 3000,220 3000,221

In [ ]:
import shutil

In [ ]:
shutil.make_archive('./data/20/std/','zip','./data/20/std/')

In [ ]:
data = data.withColumn('time',F.col('timestamp').cast('double'))

In [ ]:
df = data.select('time').distinct().toPandas()['time'].values

In [ ]:
activities = ['stationery']
window_size = 20
stream_name = 'org.md2k.feature.motionsensehrv.decoded.rightwrist.all.activity.all.three.'+str(window_size)+'.secs.'+str(activities[0])
users = CC.get_stream(stream_name).select('user').distinct().toPandas()['user'].values

In [ ]:
times = CC.get_stream(stream_name).select('timestamp').distinct().toPandas()['timestamp'].values